In [ ]:
# Run this Julia script for a minimal example of calibration
include("emulation_functions.jl")
using JLD2
using Turing
using CSV
using DataFrames
using Measures
using QuasiMonteCarlo
using GlobalSensitivity
using NCDatasets
using NetCDF
using Dates
# Set a seed for reproducibility
using Random
Random.seed!(11);

In [ ]:
# labels for plotting
labels = ["flnr", "slatop", "dleaf", "dsladlai", "leaf_long",
          "s_vc", "bbbopt", "mbbopt", "smpsc", "smpso", 
          "rholvis", "rholnir", "taulvis", "taulnir", "rhosvis",
          "rhosnir", "xl", "displar", "z0mr", "vcmaxha", "vcmaxhd", 
          "jmaxha", "jmaxhd", "roota_par", "rootb_par", "grperc"]

In [ ]:
elements = ["flnr","slatop","dleaf","dsladlai","leaf_long","s_vc","bbbopt","mbbopt","smpsc","smpso","rholvis","rholnir","taulvis",
            "taulnir","rhosvis","rhosnir","xl","displar","z0mr","vcmaxha","vcmaxhd","jmaxha","jmaxhd","roota_par","rootb_par","grperc"]
quantity_prefix = ["SH","LH","NEE","GPP"]
xlabel_str=elements
ylabel_str=quantity_prefix

In [ ]:
num_pcs = 12 # we will use 12 PCs throughout, but can change this here
N_sample =2000

In [ ]:
# Load a GP emulator
emulator_filename1 = "./emulator_GPP_US_Me2.jld2";
@load emulator_filename1 array_GP pca_fit μ_z σ_z
emulator_components1 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)
println("Done")

num_pcs = 12 # we will use 12 PCs throughout, but can change this here
num_params = emulator_components1.GPs[1].dim;
N_sample =2000

# Define quantity-of-interest function 
function qoi1(param_vals, N_PCA=num_pcs)
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components1, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
lb = [0 for i in 1:num_params]
ub = [1 for i in 1:num_params]
sampler = QuasiMonteCarlo.SobolSample()

A, B = QuasiMonteCarlo.generate_design_matrices(N_sample, lb, ub, sampler, 2)

In [ ]:
@time sobol_result_Me2_GPP = GlobalSensitivity.gsa(p -> qoi1(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Me2_GPP.S1, sobol_result_Me2_GPP.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Me2_GPP.S1))
nam = repeat(labels, outer = 2)
plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
emulator_filename2 = "./emulator_NEE_US_Me2.jld2";
@load emulator_filename2 array_GP pca_fit μ_z σ_z
emulator_components2 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi2(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components2, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_Me2_NEE= GlobalSensitivity.gsa(p -> qoi2(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Me2_NEE.S1, sobol_result_Me2_NEE.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Me2_NEE.S1))
nam = repeat(labels, outer = 2)
plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
savefig(plt, "sobol_indices_$(N_sample)_samples_US_Me2_NEE.pdf")

In [ ]:
emulator_filename3 = "./emulator_EFLX_LH_TOT_US_Me2.jld2";
@load emulator_filename3 array_GP pca_fit μ_z σ_z
emulator_components3 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi3(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components3, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_Me2_LH= GlobalSensitivity.gsa(p -> qoi3(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Me2_LH.S1, sobol_result_Me2_LH.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Me2_LH.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
savefig(plt, "sobol_indices_$(N_sample)_samples_US_Me2_LH.pdf")

In [ ]:
emulator_filename4 = "./emulator_FSH_US_Me2.jld2";
@load emulator_filename4 array_GP pca_fit μ_z σ_z
emulator_components4 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi4(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    # In this case, mean GPP
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components4, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_Me2_SH= GlobalSensitivity.gsa(p -> qoi4(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Me2_SH.S1, sobol_result_Me2_SH.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Me2_SH.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
savefig(plt, "sobol_indices_$(N_sample)_samples_US_Me2_SH.pdf")

In [ ]:
emulator_filename11 = "./emulator_ER_US_Me2.jld2";
@load emulator_filename11 array_GP pca_fit μ_z σ_z
emulator_components11 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi11(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    # In this case, mean GPP
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components11, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_Me2_ER= GlobalSensitivity.gsa(p -> qoi11(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Me2_ER.S1, sobol_result_Me2_ER.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Me2_ER.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
savefig(plt, "sobol_indices_$(N_sample)_samples_US_Me2_ER.pdf")

In [ ]:
vec3=vcat(transpose(sobol_result_Me2_LH.ST),transpose(sobol_result_Me2_SH.ST),transpose(sobol_result_Me2_NEE.ST),transpose(sobol_result_Me2_ER.ST),transpose(sobol_result_Me2_GPP.ST))

In [ ]:
quantity_prefix = ["LE","H","NEE","TER","GPP"]
ylabel_str=quantity_prefix
plt=plot(vec3, st=:heatmap, clim=(0,1),  xticks = (1:26,xlabel_str), yticks=(1:5,ylabel_str), xtickfont=font(12), 
    ytickfont=font(12), color=:Reds, colorbar_title="",xrotation=-60, margin=5mm)


In [ ]:
savefig(plt, "fig4_sobol_GSA_US_Me2_4var.pdf")

In [ ]:
emulator_filename5 = "./emulator_GPP_US_Dk3.jld2";
@load emulator_filename5 array_GP pca_fit μ_z σ_z
emulator_components5 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)
num_params = emulator_components5.GPs[1].dim;

function qoi5(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    # In this case, mean GPP
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components5, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_Dk3_GPP = GlobalSensitivity.gsa(p -> qoi5(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Dk3_GPP.S1, sobol_result_Dk3_GPP.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Dk3_GPP.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
emulator_filename6 = "./emulator_GPP_US_Blo.jld2";
@load emulator_filename6 array_GP pca_fit μ_z σ_z
emulator_components6 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi6(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    # In this case, mean GPP
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components6, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_Blo_GPP = GlobalSensitivity.gsa(p -> qoi6(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_Blo_GPP.S1, sobol_result_Blo_GPP.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_Blo_GPP.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
emulator_filename7 = "./emulator_GPP_CA_TP4.jld2";
@load emulator_filename7 array_GP pca_fit μ_z σ_z
emulator_components7 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi7(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    # In this case, mean GPP
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components7, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_TP4_GPP = GlobalSensitivity.gsa(p -> qoi7(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_TP4_GPP.S1, sobol_result_TP4_GPP.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_TP4_GPP.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
emulator_filename8 = "./emulator_GPP_US_NR1.jld2";
@load emulator_filename8 array_GP pca_fit μ_z σ_z
emulator_components8 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z)

function qoi8(param_vals, N_PCA=num_pcs)
    # A function to compute a QOI
    # In this case, mean GPP
    θ = hcat(param_vals...)
    emulator_eval = calibration_emulator(θ, emulator_components8, N_PCA)
    return mean(emulator_eval)
end

In [ ]:
@time sobol_result_NR1_GPP = GlobalSensitivity.gsa(p -> qoi8(p, num_pcs), GlobalSensitivity.Sobol(order=[0,1,2]), A, B)

In [ ]:
fillrange = 0.01
z = collect(zip(sobol_result_NR1_GPP.S1, sobol_result_NR1_GPP.ST))
mat = [tup[k] for tup in z, k in 1:2]

ctg = repeat(["First-Order", "Total"], inner = length(sobol_result_NR1_GPP.S1))
nam = repeat(labels, outer = 2)

plt = groupedbar(mat, group = ctg, spacing=0.5, ylabel = "Sobol Index",
            title = "", linecolor = [:magenta :blue], framestyle = :box, ylim=(0,1),  xticks = (1:26,xlabel_str),
            xtickfontsize=10, ylabelfontsize=18,bar_width=0.8, xrotation=90, widen=false, grid=false,
            tick_direction=:none,c = [:magenta :blue],legend_background_color=:white, 
            legend_foreground_color=:white, margin=5mm)

In [ ]:
elements = ["flnr","slatop","dleaf","dsladlai","leaf_long","s_vc","bbbopt","mbbopt","smpsc","smpso","rholvis","rholnir","taulvis",
            "taulnir","rhosvis","rhosnir","xl","displar","z0mr","vcmaxha","vcmaxhd","jmaxha","jmaxhd","roota_par","rootb_par","grperc"]
site_prefix = ["US-Dk3","CA-TP4","US-NR1","US-Blo","US-Me2"]
xlabel_str=elements
ylabel_str=site_prefix 

In [ ]:
vec2=vcat(transpose(sobol_result_Me2_GPP.ST),transpose(sobol_result_Blo_GPP.ST),transpose(sobol_result_NR1_GPP.ST),transpose(sobol_result_TP4_GPP.ST),transpose(sobol_result_Dk3_GPP.ST))

In [ ]:
plt=plot(vec2, st=:heatmap, clim=(0,1),  xticks = (1:26,xlabel_str), yticks=(1:5,ylabel_str), xtickfont=font(12), 
    ytickfont=font(12),  color=:Reds, colorbar_title="",xrotation=-60, margin=5mm)

In [ ]:
savefig(plt, "fig5_sobol_Sobol_GSA_GPP_5sites.pdf")